In [1]:
import chess
import chess.engine
import random
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10200472492081303375
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6254755840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11956849656963075820
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [5]:
model = build_model(32, 4)
utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


Skip connections (residual network) will likely improve the model for deeper connections. If you want to test the residual model, check the code below.

In [6]:
def build_model_residual(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(board3d)
  for _ in range(conv_depth):
    previous = x
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, previous])
    x = layers.Activation('relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

# It's training time!

In [7]:
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = np.load(r'C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\data\random_generated\dataset.npz')
	b, v = container['b'], container['v']
	v = np.asarray(v / abs(v).max() / 2 + 0.5, dtype=np.float32) # normalization (0 - 1)
	return b, v


x_train, y_train = get_dataset()
print(x_train.shape)
print(y_train.shape)

(1500000, 14, 8, 8)
(1500000,)


In [8]:
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d (Conv2D)             (None, 32, 8, 8)          4064      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 8, 8)          9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 8, 8)          9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 8, 8)          9248      
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                131136

In [9]:
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save('./engine_models/basic_model.h5')

Epoch 1/1000
660/660 [==============================] - 17s 15ms/step - loss: 0.0012 - val_loss: 6.9362e-04 - lr: 5.0000e-04
Epoch 2/1000
660/660 [==============================] - 9s 14ms/step - loss: 5.7919e-04 - val_loss: 5.4775e-04 - lr: 5.0000e-04
Epoch 3/1000
660/660 [==============================] - 9s 14ms/step - loss: 4.7209e-04 - val_loss: 5.0296e-04 - lr: 5.0000e-04
Epoch 4/1000
660/660 [==============================] - 9s 14ms/step - loss: 4.2278e-04 - val_loss: 4.6515e-04 - lr: 5.0000e-04
Epoch 5/1000
660/660 [==============================] - 9s 14ms/step - loss: 3.9067e-04 - val_loss: 4.1196e-04 - lr: 5.0000e-04
Epoch 6/1000
660/660 [==============================] - 9s 14ms/step - loss: 3.7008e-04 - val_loss: 4.2994e-04 - lr: 5.0000e-04
Epoch 7/1000
660/660 [==============================] - 9s 14ms/step - loss: 3.4864e-04 - val_loss: 3.6814e-04 - lr: 5.0000e-04
Epoch 8/1000
660/660 [==============================] - 9s 14ms/step - loss: 3.3617e-04 - val_loss: 3.4716e